In [2]:
import pandas as pd
import numpy as np

telco = pd.read_csv(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\data\altdata\telco.csv")
edu = pd.read_csv(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\data\altdata\StudentsPerformance.csv")
wallet = pd.read_csv(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\data\altdata\Digital_Wallet_Transactions.csv")

print("Telco:", telco.shape)
print("Education:", edu.shape)
print("Wallet:", wallet.shape)

Telco: (7043, 21)
Education: (1000, 8)
Wallet: (5000, 16)


In [3]:
# Telco Data Processing
# Clean and convert data
telco['TotalCharges'] = pd.to_numeric(telco['TotalCharges'], errors='coerce')
telco = telco.dropna(subset=['TotalCharges'])

# Create FICO-like features
telco['usage_ratio'] = telco['MonthlyCharges'] / telco['TotalCharges']
telco['tenure_score'] = telco['tenure'] / telco['tenure'].max()

# Payment stability proxy
telco['spending_consistency_telco'] = telco['MonthlyCharges'] / telco['MonthlyCharges'].max()

# Label
telco['default_label'] = telco['Churn'].map({'Yes':1, 'No':0})

telco_features = telco[['customerID','usage_ratio','tenure_score','spending_consistency_telco','default_label']]
telco_features.head()

,customerID,usage_ratio,tenure_score,spending_consistency_telco,default_label
0,7590-VHVEG,1.000000,0.013889,0.251368,0
1,5575-GNVDE,0.030140,0.472222,0.479579,0
2,3668-QPYBK,0.497920,0.027778,0.453474,1
3,7795-CFOCW,0.022980,0.625000,0.356211,0
4,9237-HQITU,0.466205,0.027778,0.595368,1


In [27]:
# Wallet Data Processing
wallet['transaction_date'] = pd.to_datetime(wallet['transaction_date'])
print(wallet['transaction_date'].min(), wallet['transaction_date'].max())
print(wallet['transaction_date'].dt.date.value_counts().head(10))

2023-08-19 03:32:00 2024-08-18 01:15:00
transaction_date
2023-09-23    27
2024-03-17    26
2024-05-19    25
2024-06-20    23
2023-10-01    23
2024-07-16    23
2023-08-19    23
2024-03-31    23
2023-09-12    22
2024-03-05    22
Name: count, dtype: int64


In [ ]:
# Feature Engineering
wallet['transaction_date'] = pd.to_datetime(wallet['transaction_date'])
wallet['day'] = wallet['transaction_date'].dt.to_period('D')

# Spending Consistency (inverse of std deviation of daily spend)
daily_spend = wallet.groupby(['user_id','day'])['product_amount'].sum().reset_index()
std_spend = daily_spend.groupby('user_id')['product_amount'].std().reset_index(name='spending_std')
std_spend['spending_std'] = std_spend['spending_std'].fillna(0)
std_spend['spending_consistency_wallet'] = 1 / (1 + std_spend['spending_std'])

# Usage Level vs Limit (proxy = avg spend-to-fee ratio)
wallet['spend_to_fee'] = wallet['product_amount'] / (wallet['transaction_fee'] + 1e-6)
spend_fee = wallet.groupby('user_id')['spend_to_fee'].mean().reset_index(name='usage_level_vs_limit_wallet')

# Recent Activity Changes (robust version)
# Count transactions per day for each user
user_spending = wallet.groupby('user_id')['product_amount'].agg(['mean','std']).reset_index()
user_spending['std'] = user_spending['std'].fillna(0)
user_spending['recent_activity_change_wallet'] = 1 / (1 + (user_spending['std'] / (user_spending['mean'] + 1e-6)))
activity = user_spending[['user_id','recent_activity_change_wallet']]

# Compute per-user mean and std of daily activity
diversity = wallet.groupby('user_id').agg({
    'product_category': 'nunique',
    'merchant_name': 'nunique',
    'payment_method': 'nunique'
}).reset_index()
diversity['digital_diversity_wallet'] = (
    diversity['product_category'] + diversity['merchant_name'] + diversity['payment_method']
) / 3

# Stability ratio (higher means more consistent)
success = wallet.groupby('user_id')['transaction_status'].apply(
    lambda x: (x == 'Successful').mean()
).reset_index(name='success_rate_wallet')

# Digital Diversity (unique categories, merchants, and payment methods)
diversity = wallet.groupby('user_id').agg({
    'product_category': 'nunique',
    'merchant_name': 'nunique',
    'payment_method': 'nunique'
}).reset_index()
diversity['digital_diversity_wallet'] = (
    diversity['product_category'] + diversity['merchant_name'] + diversity['payment_method']
) / 3

# Reliability (Success Rate)
success = wallet.groupby('user_id')['transaction_status'].apply(
    lambda x: (x == 'Successful').mean()
).reset_index(name='success_rate_wallet')

# Combine all engineered features
wallet_features = (
    std_spend[['user_id','spending_consistency_wallet']]
    .merge(spend_fee, on='user_id', how='left')
    .merge(activity, on='user_id', how='left')
    .merge(diversity[['user_id','digital_diversity_wallet']], on='user_id', how='left')
    .merge(success, on='user_id', how='left')
)

wallet_features.fillna(0, inplace=True)

print("Wallet features ready:", wallet_features.shape)
wallet_features.head()

Wallet features ready: (3932, 6)


,user_id,spending_consistency_wallet,usage_level_vs_limit_wallet,recent_activity_change_wallet,digital_diversity_wallet,success_rate_wallet
0,USER_00001,1.000000,159.370539,1.000000,1.0,1.0
1,USER_00002,1.000000,78.575261,1.000000,1.0,1.0
2,USER_00019,0.000602,94.699289,0.620042,2.0,1.0
3,USER_00020,1.000000,170.386056,1.000000,1.0,1.0
4,USER_00025,1.000000,297.522043,1.000000,1.0,1.0


In [19]:
# Education Data Processing
edu = edu.rename(columns={'math score':'math','reading score':'reading','writing score':'writing'})
edu['education_index'] = edu[['math','reading','writing']].mean(axis=1) / 100
edu_features = edu[['education_index']]
edu_features.head()


,education_index
0,0.726667
1,0.823333
2,0.926667
3,0.493333
4,0.763333


In [38]:
# Align sample sizes
n = min(len(telco_features), len(wallet_features), len(edu_features))
telco_sample = telco_features.sample(n=n, random_state=42).reset_index(drop=True)
wallet_sample = wallet_features.sample(n=n, random_state=42).reset_index(drop=True)
edu_sample = edu_features.sample(n=n, replace=True, random_state=42).reset_index(drop=True)

# Combine
final = pd.concat([telco_sample, wallet_sample, edu_sample], axis=1)
final.head()

,customerID,usage_ratio,tenure_score,spending_consistency_telco,default_label,user_id,spending_consistency_wallet,usage_level_vs_limit_wallet,recent_activity_change_wallet,digital_diversity_wallet,success_rate_wallet,education_index
0,6614-YWYSC,0.016647,0.847222,0.210526,0,USER_04665,1.00000,98.432921,1.00000,1.0,1.0,0.883333
1,9546-KDTRB,0.053021,0.263889,0.208000,0,USER_03723,1.00000,230.863314,1.00000,1.0,1.0,0.503333
2,0871-URUWO,0.075239,0.180556,0.861053,1,USER_05245,0.00035,244.242721,0.69322,2.0,1.0,0.560000
3,5151-HQRDG,0.027108,0.513889,0.463579,0,USER_02929,1.00000,643.079716,1.00000,1.0,1.0,0.643333
4,6624-JDRDS,0.182409,0.083333,0.248000,0,USER_09588,1.00000,517.306261,1.00000,1.0,1.0,0.956667


In [39]:
final['User_Spending_Consistency'] = (
    final['spending_consistency_wallet'] * 0.7 + final['spending_consistency_telco'] * 0.3
)

final['Usage_Level_vs_Limit'] = final['usage_ratio']
final['Service_Tenure'] = final['tenure_score']
final['Recent_Activity_Changes_wallet'] = final['recent_activity_change_wallet']
final['Diversity_of_Digital_Behavior'] = final['education_index']

final = final[[
    'User_Spending_Consistency',
    'Usage_Level_vs_Limit',
    'Service_Tenure',
    'Recent_Activity_Changes_wallet',
    'Diversity_of_Digital_Behavior',
    'default_label'
]]
final.head()

,User_Spending_Consistency,Usage_Level_vs_Limit,Service_Tenure,Recent_Activity_Changes_wallet,Diversity_of_Digital_Behavior,default_label
0,0.763158,0.016647,0.847222,1.00000,0.883333,0
1,0.762400,0.053021,0.263889,1.00000,0.503333,0
2,0.258561,0.075239,0.180556,0.69322,0.560000,1
3,0.839074,0.027108,0.513889,1.00000,0.643333,0
4,0.774400,0.182409,0.083333,1.00000,0.956667,0


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled = pd.DataFrame(
    scaler.fit_transform(final.drop(columns=['default_label'])),
    columns=final.columns[:-1]
)
scaled['default_label'] = final['default_label']

scaled.to_csv("C:\\Users\\Asus\\Documents\\GitHub\\Credit-Scoring\\data\\altdata\\Hybrid_CreditScoring_AlternativeData.csv", index=False)
print("Final dataset saved — shape:", scaled.shape)
scaled.head()

Final dataset saved — shape: (1000, 6)


,User_Spending_Consistency,Usage_Level_vs_Limit,Service_Tenure,Recent_Activity_Changes_wallet,Diversity_of_Digital_Behavior,default_label
0,0.751475,0.004095,0.845070,1.000000,0.857143,0
1,0.750679,0.040934,0.253521,1.000000,0.391837,0
2,0.221140,0.063435,0.169014,0.474918,0.461224,1
3,0.831263,0.014690,0.507042,1.000000,0.563265,0
4,0.763291,0.171973,0.070423,1.000000,0.946939,0
